In [1]:
from tools.project import INPUT_PATH, LOGS_PATH, OUTPUT_PATH, RAW_PATH
import torch
import torch
from audiocraft.data.audio import audio_read, audio_write
from audiocraft.data.audio_utils import convert_audio_channels, convert_audio
import numpy as np
from audioldm_eval.metrics.fad import FrechetAudioDistance
import os
DEVICE = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
EXAMPLES_LEN = 5
torch.cuda.is_available()

True

In [3]:
# model = MusicGen.get_pretrained('facebook/musicgen-small')
# model.set_generation_params(
# 	use_sampling=True,
# 	top_k=250,
# 	duration=EXAMPLES_LEN
# )

Using cache found in /home/mszawerda/.cache/torch/hub/harritaylor_torchvggish_master


Reload fad_generated_folder_cache /home/mszawerda/musical-generative-models-conditioning/data/input/textual-inversion/metal_fad_fad_feature_cache.npy
Loading data to RAM


100%|██████████| 14/14 [00:00<00:00, 237.22it/s]


{'frechet_audio_distance': 30.598606179254183}

In [30]:
res=model.generate([f"music in the style of jazz song"]*5, progress=True)
for a_idx in range(res.shape[0]):
    music = res[a_idx].cpu()
    music = music/np.max(np.abs(music.numpy()))
    path = OUTPUT_PATH("textual-inversion", 'metal', 'temp', f'music_p{a_idx}')
    audio_write(path, music, model.cfg.sample_rate)

In [3]:
fad = FrechetAudioDistance(verbose=True, use_pca=False, use_activation=False)
fad.model = fad.model.to(DEVICE)

f = lambda x, y: f"({x}, {y}): {fad.score(OUTPUT_PATH('textual-inversion-v3', 'metal', 'temp'), RAW_PATH('fad_debug', y))['frechet_audio_distance']}"
f('3', '1')

Using cache found in /home/mszawerda/.cache/torch/hub/harritaylor_torchvggish_master


Reload fad_generated_folder_cache /home/mszawerda/musical-generative-models-conditioning/data/output/textual-inversion-v3/metal/temp_fad_feature_cache.npy
Reload fad_target_folder_cache /home/mszawerda/musical-generative-models-conditioning/data/raw/fad_debug/1_fad_feature_cache.npy


'(3, 1): 19.453124712779925'

In [11]:
# fd_score = fad.score(INPUT_PATH('textual-inversion-v3', 'data', 'train', 'cluster_0', 'audio'), OUTPUT_PATH("textual-inversion", 'metal', 'temp'))
# fd_score = fad.score(INPUT_PATH('textual-inversion-v3', 'data', 'train', 'cluster_0', 'audio'), OUTPUT_PATH('textual-inversion-v3', 'cluster_10', 'temp'))
fd_score = fad.score(OUTPUT_PATH('textual-inversion-v3', 'cluster_2', 'temp'), OUTPUT_PATH('textual-inversion-v3', 'cluster_10', 'temp'), recalculate=True)
# fd_score = fad.score(INPUT_PATH('textual-inversion-v3', 'data', 'train', 'cluster_0', 'audio'), INPUT_PATH('textual-inversion-v3', 'data', 'train', 'cluster_1', 'audio'))
print(fd_score)
# os.remove(OUTPUT_PATH("textual-inversion", 'metal', 'temp_fad_feature_cache.npy'))
# list(fd_score.values())[0]

Loading data to RAM


100%|██████████| 5/5 [00:00<00:00, 158.28it/s]


Loading data to RAM


100%|██████████| 5/5 [00:00<00:00, 191.71it/s]


[Frechet Audio Distance] exception thrown, Imaginary component 0.25250462336960655
-1
